In [1]:
from __future__ import print_function

# pytorch import and library calls

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable


# tensorflow imports
import tensorflow as tf

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [20]:
''' Reference of multiplication and operations of tensors in pytorch'''

'''x_tensor = torch.Tensor([[1, 2], [3, 4]])
y_tensor = torch.Tensor([[5, 6], [7, 8]])

x_variable = torch.autograd.Variable(x_tensor)

print(x_tensor * y_tensor)
print(x_variable * 2)
print(x_variable * y_tensor)
print(((x_tensor + y_tensor))/4.0)
'''

'x_tensor = torch.Tensor([[1, 2], [3, 4]])\ny_tensor = torch.Tensor([[5, 6], [7, 8]])\n\nx_variable = torch.autograd.Variable(x_tensor)\n\nprint(x_tensor * y_tensor)\nprint(x_variable * 2)\nprint(x_variable * y_tensor)\nprint(((x_tensor + y_tensor))/4.0)\n'

In [ ]:
batchSize = 4
imageSize = 256
#transforms.RandomCrop(imageSize),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#transforms.ToTensor()
#transforms.ToPILImage(),
data_transform = transforms.Compose([transforms.Resize(imageSize),transforms.ToTensor()]
                                   )

In [ ]:
#data_transform

In [ ]:
#check for our GPU, otherwise, use a CPU

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else: 
    #SUN LAB AND NON-GPU PCs
    DEVICE = torch.device('cpu')

In [ ]:
ROOT_DATA_FOLDER = '/home/peo5032/Documents/COMP594'
NON_URBAN_DATA_FOLDER = '/home/peo5032/Documents/COMP594/NON_URBAN_IMAGES'
URBAN_DATA_FOLDER = '/home/peo5032/Documents/COMP594/URBAN_IMAGES'


ROOT_TEST_FOLDER = '/home/peo5032/Documents/COMP594_TEST'
TEST_URBAN_DATA_FOLDER = '/home/peo5032/Documents/COMP594/URBAN_IMAGES'

In [ ]:
#non_urban_dataset = datasets.ImageFolder(root=NON_URBAN_DATA_FOLDER ,
#                                           transform=data_transform)

urban_dataset = datasets.ImageFolder(root = ROOT_DATA_FOLDER, 
                                     transform=data_transform)

In [ ]:
data_loader = torch.utils.data.DataLoader(urban_dataset,
                                             batch_size=batchSize, shuffle=True,
                                             num_workers=1)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02).to(device=DEVICE)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02).to(device=DEVICE)
        m.bias.data.fill_(0).to(device=DEVICE)


In [ ]:
class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 300, 10, 1, 0, bias = False),
            nn.BatchNorm2d(300),
            nn.ReLU(True),
            nn.ConvTranspose2d(300, 256, 10, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 10, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 256, 10, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 3, 10, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input.to(device=DEVICE))
        return output.to(device=DEVICE)
    
netG = G().to(device=DEVICE)
netG.apply(weights_init).to(device=DEVICE)

In [ ]:
class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 256, 10, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 128, 10, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 512, 10, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 256, 10, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 1, 10, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input.to(device=DEVICE))
        return output.view(-1).to(device=DEVICE)

netD = D()
netD.apply(weights_init).to(device=DEVICE)

In [ ]:
'''
criterion = nn.BCELoss().to(device=DEVICE)
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))
epochBound = 30

for epoch in range(epochBound):

    for i, data in enumerate(data_loader, 0):
        
        netD.zero_grad()
        
        real, _ = data
        input = Variable(real)
        #print("INPUT IS:", input)
        target = Variable(torch.ones(input.size()[0]).to(device=DEVICE)).to(device=DEVICE)
        #print("TAGET IS:", target)
        output = netD(input).to(device=DEVICE)
        #print("OUTPUT IS:", output)
        #target = Variable(torch.ones(output.size()[0]))
        errD_real = criterion(output, target).to(device=DEVICE)
        
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1).to(device=DEVICE))
        fake = netG(noise).to(device=DEVICE)
        target = Variable(torch.zeros(input.size()[0]).to(device=DEVICE))
        output = netD(fake.detach()).to(device=DEVICE)
        errD_fake = criterion(output, target)
        
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]).to(device=DEVICE)).to(device=DEVICE)
        output = netD(fake).to(device=DEVICE)
        #arget = Variable(torch.ones(output.size()[0]))
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, epochBound, i, len(data_loader), errD.data[0], errG.data[0]))
        if i % 50 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./GANitR1", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./", epoch), normalize = True)
            '''

In [ ]:
import os
os.getcwd()